In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from scipy import stats
pd.set_option('display.max_columns', None)
from scipy.stats import skew, kurtosis
import pygris
from shapely.geometry import Polygon
import shapely
from inconsistency_utils import *

In [2]:
#Read the original dataset

df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

## Sixth Dataset [censusBlockGroupFips and Tract NA (Missing together) and NA Zipcode]

In [3]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [4]:
# Defining conditions
condition1 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].isna() & 
              df_geographic_unique['countyCode'].notna() & 
              df_geographic_unique['reportedZipCode'].isna())

condition2 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].isna() & 
              df_geographic_unique['countyCode'].notna() & 
              df_geographic_unique['reportedZipCode'].notna() &
              (df_geographic_unique['yearOfLoss'] < 1980))

# Filtering the dataframe based on the combined conditions
df_geographic_unique = df_geographic_unique[condition1 | condition2]

In [5]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss
91262,FL,NaN,12113.0,NaN,NaN,30.4,-86.9,1995
413984,AL,NaN,1003.0,NaN,NaN,30.5,-87.9,2004
413985,AL,NaN,1003.0,NaN,NaN,30.5,-87.9,2005
437317,NC,NaN,37129.0,NaN,NaN,34.2,-77.8,1996
437318,NC,NaN,37129.0,NaN,NaN,34.2,-77.8,1998


In [6]:
print(sum(df_geographic_unique['state'].isna()))
print(sum(df_geographic_unique['reportedZipCode'].isna()))
print(sum(df_geographic_unique['countyCode'].isna()))
print(sum(df_geographic_unique['censusTract'].isna()))
print(sum(df_geographic_unique['censusBlockGroupFips'].isna()))
print(sum(df_geographic_unique['latitude'].isna()))
print(sum(df_geographic_unique['yearOfLoss'].isna()))

0
349
0
5165
5165
0
0


In [7]:
#convert units to string type to easily map

columns_to_format = {
    'countyCode': 5
}

df_geographic_unique = format_geographic_units(df_geographic_unique, columns_to_format)

In [8]:
# Creating year bins to map (NEED TO UPDATE IT ONCE WE HAVE 2023 ZIPCODE SHAPEFILE DATA AND THE LATEST ORIGINAL FEMA DATASET) 

bins_1980_2021 = [df['yearOfLoss'].min(), 1980, 1990, 2000, 2010, 2020, df['yearOfLoss'].max() + 1]
labels_1980_2021 = [0, 1980, 1990, 2000, 2010, 2020]

custom_bins = [0, 2000] + list(range(2010, 2023)) + [2024]
custom_labels = [0, 2000] + list(range(2010, 2023))

df_geographic_unique = process_year_of_loss(df_geographic_unique, bins_1980_2021, labels_1980_2021, custom_bins, custom_labels)

In [9]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1980_2021,zip_year_bin
91262,FL,NaN,12113,NaN,NaN,30.4,-86.9,1995,1990,0
413984,AL,NaN,01003,NaN,NaN,30.5,-87.9,2004,2000,2000
413985,AL,NaN,01003,NaN,NaN,30.5,-87.9,2005,2000,2000
437317,NC,NaN,37129,NaN,NaN,34.2,-77.8,1996,1990,0
437318,NC,NaN,37129,NaN,NaN,34.2,-77.8,1998,1990,0


## Read shapefiles

In [10]:
units = ['state', 'lat_long', 'County']
base_path = "C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/"
df_shapefiles = load_and_rename_geographic_data_whole(units, base_path)

state_df = df_shapefiles['state']
lat_long_df = df_shapefiles['lat_long']
County_df = df_shapefiles['County']

Using the default year of 2021


C:\Users\Asus\Box\Home Folder lk220\Private\Duke\Duke Courses\Research\FloodDamagePrediction\Code\inconsistency_utils.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)


## Geometry Intersection creation

In [13]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [14]:
# Setting the multi-index on lat_long_df
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

# Resetting the index of lat_long_df (optional, but good practice)
state_df.reset_index(inplace=True)

In [15]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

#### Drop rows with missing shapefiles

In [16]:
df_geographic_unique = df_geographic_unique[(df_geographic_unique['geometry_county'].notna())]

#### Creating the intersections

In [17]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'latitude', 'longitude', 'year', 'state',
                                       'geometry_county','geometry_lat_long','geometry_state',
                                         'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 
                                        'stateInconsistent', 'latlongInconsistent', 'multiple', 'noOverlap', 'oneWrong'])

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1980_2021']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode = row_unit['reportedZipCode']
    
    # Compute intersection geometry
    intersection_geometry = lat_long_geometry.intersection(county_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'state': [state],
            'geometry_county': [county_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'geometry_state': [state_geometry]
        })], ignore_index=True)

In [18]:
len(new_unit_df)

0

In [19]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = state_geometry.intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 1

In [20]:
new_unit_df['countyInconsistent'].sum()

0

In [21]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode = row_unit['reportedZipCode']   

    intersection_geometry = county_geometry.intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 1

In [22]:
new_unit_df['stateInconsistent'].sum()

0

In [23]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = county_geometry.intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 1

In [24]:
new_unit_df['latlongInconsistent'].sum()

0

In [25]:
mask = (new_unit_df['latlongInconsistent'] +   new_unit_df['stateInconsistent'] + new_unit_df['countyInconsistent'] ) > 1
new_unit_df.loc[mask, 'multiple'] = 1
new_unit_df.loc[~mask, 'multiple'] = 0

print(sum(new_unit_df['multiple']))

0


In [26]:
new_unit_df.head()

,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,year,state,geometry_county,geometry_lat_long,geometry_state,cbgInconsistent,tractInconsistent,countyInconsistent,stateInconsistent,latlongInconsistent,multiple,noOverlap,oneWrong


In [27]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent']   ) == 0

new_unit_df['noOverlap'] = 0  # Default to 0
new_unit_df.loc[condition, 'noOverlap'] = 1

print(sum(new_unit_df['noOverlap']))

0


In [28]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] )  == 1

new_unit_df['oneWrong'] = 0  # Default to 0
new_unit_df.loc[condition, 'oneWrong'] = 1

print(sum(new_unit_df['oneWrong']))


0


In [29]:
# Initialize the count dictionary for 'Wrong' values
wrong_counts = {
    'lat_long_geometry': 0,
    'state_geometry': 0,
    'county_geometry': 0,
    'tract_geometry': 0,
    'bg_geometry': 0,
    'zip_geometry': 0  # added for zip codes
}

# Iterate through the DataFrame
for idx_unit, row_unit in new_unit_df.iterrows():
    if row_unit['oneWrong'] == 1:
        if row_unit['latlongInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'lat_long_geometry'] = 'Wrong'
            wrong_counts['lat_long_geometry'] += 1
        if row_unit['stateInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'state_geometry'] = 'Wrong'
            wrong_counts['state_geometry'] += 1
        if row_unit['countyInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'county_geometry'] = 'Wrong'
            wrong_counts['county_geometry'] += 1
        if row_unit['tractInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'tract_geometry'] = 'Wrong'
            wrong_counts['tract_geometry'] += 1
        if row_unit['cbgInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'bg_geometry'] = 'Wrong'
            wrong_counts['bg_geometry'] += 1
        if row_unit['zipInconsistent'] == 1:  # added for zip codes
            new_unit_df.at[idx_unit, 'zip_geometry'] = 'Wrong'
            wrong_counts['zip_geometry'] += 1

# Print the 'Wrong' counts
for category, count in wrong_counts.items():
    print(f"{category}: {count} 'Wrong' values")

lat_long_geometry: 0 'Wrong' values
state_geometry: 0 'Wrong' values
county_geometry: 0 'Wrong' values
tract_geometry: 0 'Wrong' values
bg_geometry: 0 'Wrong' values
zip_geometry: 0 'Wrong' values


In [30]:
columns_to_drop = [
    'geometry_county', 'geometry_state', 'geometry_lat_long'
]

# Drop the columns
new_unit_df = new_unit_df.drop(columns= columns_to_drop)

In [31]:
new_unit_df= new_unit_df.drop_duplicates()

new_unit_df.to_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/inconsistency_dataframe_6.parquet.gzip', compression = 'gzip',  index=False)